In [1]:
!pip install --upgrade --user google-cloud-aiplatform google-cloud-bigquery

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 62.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.1/240.1 kB 26.7 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [1]:
# Get project ID
PROJECT_ID = ! gcloud config get-value project
PROJECT_ID = PROJECT_ID[0]
LOCATION = "us-central1" # @param {type:"string"}
print(PROJECT_ID)

qwiklabs-gcp-02-6dae406f4bf4


In [2]:
from google.cloud import aiplatform
aiplatform.init(project=PROJECT_ID, location=LOCATION)

print("Initialized")

Initialized


In [3]:
import vertexai
from vertexai.generative_models import GenerativeModel, Part

def answer_question_gemini(prompt):
  model = GenerativeModel("gemini-pro")
  response = model.generate_content(
    prompt,
    generation_config={
        "max_output_tokens": 8192,
        "temperature": 0.5,
        "top_p": 0.5,
        "top_k": 10,
    },
  stream=False,
  )
  try:
    return response.text
  except:
    print("An Error Ocuured Cleaning the Data")
    return "An Error Ocuured Cleaning the Data"

In [4]:
def run_search(question):
  from google.cloud import bigquery

  client = bigquery.Client()

  sql = """
      SELECT query.query, base.title, base.body
      FROM VECTOR_SEARCH(
      TABLE `bbc_news.bbc_news_with_embeddings`, 'text_embedding',
      (
      SELECT text_embedding, content AS query
      FROM ML.GENERATE_TEXT_EMBEDDING(MODEL `bbc_news.bq_embedding_model`,
          (SELECT @question AS content))),
      top_k => 5)
      """

  job_config = bigquery.QueryJobConfig(
    query_parameters=[
        bigquery.ScalarQueryParameter("question", "STRING", question),
    ]
  )

  query_job = client.query(sql, job_config=job_config)

  data = ""
  for row in query_job:
    data += row.body + "\n"

  return data

In [5]:
def build_prompt(data, question):
  prompt = """
    Instructions: Answer the question using the following Context.

    Context: {0}

    Question: {1}
  """.format(data, question)
  return prompt

In [6]:
from IPython.core.display import display, HTML

def answer_question(question):

  data = run_search(question)
  display("Retrieved Data:")
  display(data)
  display(" . . . ")
  prompt = build_prompt(data, question)
  answer_gemini = answer_question_gemini(prompt)

  return answer_gemini

In [8]:
QUESTION = "What's happening in sports?"

answer_gemini = answer_question(QUESTION)
display("User Question:")
display(QUESTION)
display("--------------------------------")
display("Gemini Answer:")
display(answer_gemini)

'Retrieved Data:'

'The current slew of sports games offers unparalleled opportunities for fans who like to emulate on-field action without ever moving from the couch.\n\nThe two giants in the field - ESPN and EA Sports - have been locked in a heavyweight battle for years. The latter is the world\'s largest games manufacturer. Years of experience mean that the titles in their steady flow of sport sims are finely honed, massively entertaining and ooze flair. Sports broadcaster ESPN, meanwhile, has leant its name to a series of games that are similarly classy but lower in profile and price. But that status quo was changed forever - or for the next 15 years at least - by a deal earlier this year when ESPN sold EA the rights to its TV branding and on-air talent, meaning the ESPN games presently developed by Sega will have to come to and end in their present form.\n\nIt was a massive-money deal that not only raised eyebrows but stirred active indignation in many quarters, with fans concerned that it is set to

' . . . '

'User Question:'

"What's happening in sports?"

'--------------------------------'

'Gemini Answer:'

"## Sports News in 2004:\n\n**Athletics:**\n\n* **Kelly Holmes:** Won gold in the 800m and 1500m at the Athens Olympics, becoming the first Briton to achieve the Olympic middle-distance double in 84 years.\n* **Paula Radcliffe:** Dropped out of the marathon and 10,000m at the Athens Olympics due to exhaustion. However, she won the New York Marathon in November.\n* **British men's 4x100m relay team:** Won gold at the Athens Olympics, their first relay medal since 1912.\n* **Kelly Sotherton:** Won bronze in the heptathlon at the Athens Olympics.\n* **Steve Backley:** Retired from javelin throwing after finishing fourth at the Athens Olympics.\n* **Dean Macey:** Returned from injury to finish fourth in the decathlon at the Athens Olympics.\n* **Drugs scandals:** Several athletes were caught using performance-enhancing drugs, including Kenteris, Thanou, Annus, Fazelas, Korzhanenko, Chambers, and White.\n\n**Other sports:**\n\n* **Hicham El Guerrouj:** Won gold in the 1500m and 5,000m at th